In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.recommendation import ALS

In [4]:
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
dataset = spark.read.csv('movielens_ratings.csv', inferSchema=True, header=True)

In [6]:
dataset.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
dataset.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [8]:
dataset.select('userId').distinct().show()

+------+
|userId|
+------+
|    28|
|    26|
|    27|
|    12|
|    22|
|     1|
|    13|
|     6|
|    16|
|     3|
|    20|
|     5|
|    19|
|    15|
|     9|
|    17|
|     4|
|     8|
|    23|
|     7|
+------+
only showing top 20 rows



In [9]:
training, test = dataset.randomSplit([0.8, 0.2])

In [10]:
als = ALS(maxIter=10, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [11]:
model = als.fit(training)

In [12]:
predictions = model.transform(test)

In [13]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    27| 0.25881585|
|     31|   4.0|    12| 0.17348808|
|     31|   1.0|     5| 0.45591387|
|     31|   1.0|     4| 0.07460113|
|     85|   1.0|    15|  1.7251365|
|     85|   3.0|    21|  2.9670918|
|     65|   1.0|    19|  1.3692412|
|     65|   5.0|    23|  2.0647616|
|     53|   2.0|    19|   0.970651|
|     78|   1.0|    19| 0.66156983|
|     78|   1.0|    11|   1.015026|
|     34|   3.0|     3| -1.6523411|
|     34|   3.0|    25| -0.6739404|
|     81|   3.0|    26|-0.09985408|
|     81|   1.0|     3|  2.8363864|
|     81|   1.0|    15|  1.3031683|
|     28|   3.0|     1|  -0.232141|
|     28|   1.0|    23|  0.5466939|
|     28|   1.0|    14| 0.91793835|
|     28|   1.0|     2|  2.7799096|
+-------+------+------+-----------+
only showing top 20 rows



In [14]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [15]:
rmse = evaluator.evaluate(predictions)

In [16]:
print ('RMSE')
print (rmse)

RMSE
1.5333353792011613


In [18]:
single_user = test.filter(test['userId']==11).select(['movieId', 'userId'])

In [19]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     20|    11|
|     22|    11|
|     36|    11|
|     47|    11|
|     50|    11|
|     67|    11|
|     70|    11|
|     71|    11|
|     78|    11|
|     82|    11|
|     90|    11|
|     94|    11|
|     99|    11|
+-------+------+



In [20]:
recommendations = model.transform(single_user)

In [21]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     36|    11|  4.1127057|
|     50|    11|  3.7674112|
|     94|    11|  2.2356355|
|     20|    11|  2.1962209|
|     70|    11|  1.7080048|
|     90|    11|  1.4562201|
|     71|    11|  1.3394716|
|     22|    11|  1.2550706|
|     82|    11|  1.1632495|
|     78|    11|   1.015026|
|     67|    11|  0.7237822|
|     99|    11|   0.533915|
|     47|    11|-0.10666394|
+-------+------+-----------+

